In [2]:
from requests import get
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')         # to fliter warning in notebook

In [14]:
dmd_df = pd.read_csv('D:/Military/Crawl/dmd_df_.csv')
prd_df = pd.read_csv('D:/Military/Crawl/prd_df_.csv')

## Data Prepration 

In [4]:
import nltk
from nltk.corpus import stopwords
import re
import codecs
from hazm import *
from collections import Counter

In [73]:
nmz = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
tagger = POSTagger(model='pos_tagger.model')  # Adjust the model path if necessary

nltk.download("punkt_tab")

en_stops = set(stopwords.words("english")) 
fa_stops = sorted(list(set([nmz.normalize(w) for w in codecs.open(
    'persian.txt', encoding='utf-8').read().split('\n') if w])))
tech_stops = sorted(list(set([nmz.normalize(w) for w in codecs.open(
    'technical_stop.txt', encoding='utf-8').read().split('\n') if w])))

def tokenize(df, columns):
    for c in columns:
        df = df.drop('tokenized_' + str(c), axis=1, errors='ignore')        
        df.insert(loc=df.shape[1], column='tokenized_' + str(c), value=None)
        # Process each row in the specified column
        for i in range(df.shape[0]):
            tokenized_dataset = []
            # Replace punctuation with space
            text = re.sub(r'[^\w\s]', ' ', str(df[c][i]))
            # Step 1: Normalize the text
            normalized_text = nmz.normalize(text)
            # Step 2: Tokenize the normalized text
            tokens = nltk.word_tokenize(normalized_text)
            # Step 3: POS tagging
            tagged_tokens = tagger.tag(tokens)
            # Step 4: Stem, lemmatize, and keep only nouns (tagged as 'NOUN','NOUN,EZ')
            filtered_words = [
                lemmatizer.lemmatize(stemmer.stem(word)) 
                for word, pos in tagged_tokens 
                if pos in (['NOUN','NOUN,EZ']) 
                and word not in fa_stops 
                and word.lower() not in en_stops 
                and word not in tech_stops 
                and not word.isdigit()
            ]            
            # Append the filtered words to the tokenized dataset
            tokenized_dataset.append(filtered_words)
            # Save the tokenized dataset to the new column
            df.at[i, 'tokenized_' + str(c)] = tokenized_dataset[0]
    return df

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\WALLEX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [93]:
tokenized_dmd_df = tokenize(dmd_df,['dmd_title','dmd_description','dmd_key_words'])

In [94]:
tokenized_dmd_df.to_csv('tokenized_dmd_df_.csv', encoding='utf-8', index=False)

In [95]:
tokenized_dmd_df.tail(10)

,Unnamed: 0,dmd_id,dmd_title,dmd_urlIdentifier,dmd_description,dmd_key_words,dmd_expiration_date,tokenized_dmd_title,tokenized_dmd_description,tokenized_dmd_key_words
926,926,6486847c-efd4-4ff7-bc0e-b46033e49e38,بسته بندی لوله های پلی اتیلنی شاخه ای در صنعت گاز,DMD-929,لوله های پلی اتیلن گاز پس از تولید باید در شرا...,['بسته-بندی-لوله-های-پلی-اتیلنی-شاخه-ای-در-صنع...,7/24/2025 0:00,"[لوله, پل, اتیلن, شاخه, گاز]","[لوله, پلی‌اتیلن, گاز, استاندارد, باندول, پروژ...","[لوله, پل, اتیلن, شاخه, گاز]"
927,927,eb141e9d-9cf5-4f6a-9c35-665378e2b620,نیاز به ایجاد قوام و سختی در حبه ها و جلوگیری ...,DMD-930,با توجه به انحصاری بودن فرایند تولید قند های ط...,['نیاز-به-ایجاد-قوام-و-سختی-در-حبه-ها-و-جلوگیر...,7/23/2025 0:00,"[قوا, سخت, حبه, وارفتگ, حبه, آب, ضایع]","[قند, طع, قند, قوا, سخت, حبه, قند, اقدا, نب, ش...","[قوا, سخت, حبه, وارفتگ, حبه, آب, ضایع]"
928,928,97eab808-8bc0-497f-9d96-9adc0c04851a,کاهش مصرف آب مورد مصرف جهت تولید رب گوجه که نی...,DMD-931,با توجه به مصرف زیاد آب در تولید رب گوجه نظر ب...,['کاهش-مصرف-آب'],7/30/2025 0:00,"[آب, رب, گوجه, آب, بهینه‌ساز]","[آب, رب, گوجه, صرفه‌جو, هزینه, اصلاح]",[آب]
929,929,af48d551-fc97-4ab1-8423-3e5375796da7,سیستم مانیتورینگ محصولات در زمان تولید,DMD-932,سیستم اتوماسیون داخلی جهت رصد لحظه به لحظه وضع...,['سیستم-مانیتورینگ-محصولات-در-زمان-تولید'],7/30/2025 0:00,[مانیتورینگ],"[اتوماسیون, رصد, وضع, سفار]",[مانیتورینگ]
930,930,20aea0ab-1a7e-434d-920a-4edbbb3864ad,دانش فنی تولید اددتیو روغن موتور,DMD-933,با توجه به اینکه فعالیت تولیدی شرکت در زمینه ت...,['دانش-فنی-تولید-اددتیو-روغن-موتور'],7/30/2025 0:00,"[اددتیو, روغن, موتور]","[روغن, موتور, اددتیو, روغن, موتور, دستیاب, ادد...","[اددتیو, روغن, موتور]"
931,931,7359b516-a6e8-4a18-a50f-29cf8ad8fae5,دانش فنی تولید اددتیو ضدیخ,DMD-934,با توجه به اینکه فعالیت تولیدی شرکت در زمینه ت...,['دانش-فنی-تولید-اددتیو-ضدیخ'],7/23/2025 0:00,"[اددتیو, ضدیخ]","[ضدیخ, اددتیو, ضدیخ, دستیاب, اددتیو, ضدیخ]","[اددتیو, ضدیخ]"
932,932,737f0d39-f22b-420a-bd34-738ff90e126c,طراحی دستگاه چین کن تیغه‏ای به منظور تولید فیل...,DMD-935,طراحی دستگاه چین کن تیغه‏ای به منظور تولید فیل...,['طراحی-دستگاه-چین-کن-تیغه\u200fای-به-منظور-تو...,7/23/2025 0:00,"[چین, تیغه, فیلتر]","[چین, تیغه, فیلتر, فیلتر, فیلتر, روغن, فیلتر, ...","[چین, تیغه, u, fا, فیلتر]"
933,933,1488d5a8-0225-429d-b68d-1d4f27fea8db,طراحی دستگاه تولید لایه‏ های مورد استفاده در ف...,DMD-936,طراحی دستگاه تولید لایه‏ های مورد استفاده در ف...,['طراحی-دستگاه-تولید-لایه\u200f-های-مورد-استفا...,7/22/2025 0:00,"[لایه, فیلتر, پلیتد]","[لایه, فیلتر, پلیتد, فیلتر, پلیتد, بیمارستان, ...","[لایه, u, f, فیلتر, پلیتد]"
934,934,fa35572a-a3bb-49fb-b12a-11dae3bf496c,دانش فنی یکپپارچه سازی و مدیریت تولید خط تولید...,DMD-937,با توجه به اینکه شرکت در شرف شروع فعالیت تولید...,['روغن-موتور'],7/23/2025 0:00,"[مدیر, روغن, موتور, بلندینگ]","[شرف, روغن, موتور, خریدار, تجربه, مدیر, روغن, ...","[روغن, موتور]"
935,935,3a9da060-fa5b-4993-b33c-04c7fa10d429,دانش فنی ایجاد پیوستگی و کشسانی و تنش-کرنش ژل ...,DMD-938,شرکت تولید کننده ژل جدا کننده سرح خون می باشد....,['ژل-جدا-کننده-سرم-خون'],11/3/2025 0:00,"[پیوستگ, کشسان, تن, کرن, ژل, سر, خون, مشتر]","[ژل, سرح, خون, ژل, پلیمر, وزن, ویسکوزیته, آیتم...","[ژل, سر, خون]"


In [96]:
tokenized_prd_df = tokenize(prd_df,['prd_title','prd_description','prd_key_words'])

In [97]:
tokenized_prd_df.to_csv('tokenized_prd_df_.csv', encoding='utf-8', index=False)

In [99]:
tokenized_prd_df.tail(10)

,Unnamed: 0,prd_id,prd_title,prd_urlIdentifier,prd_description,prd_key_words,prd_owner_id,prd_owner_type,prd_owner_name,tokenized_prd_title,tokenized_prd_description,tokenized_prd_key_words
16991,16991,d907a01c-52ff-42fd-b8d0-7f4613c0186b,بنزالکونیوم کلرید ۸۰,PRD-17013,-آلکیل دی متیل بنزیل آمونیوم کلرید ۸۰ یا بنزال...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[بنزالکونیو, کلرید]","[آلکیل, د, متیل, بنزیل, آمونیو, کلرید, بنزالکو...",[]
16992,16992,39581278-663c-4f4f-9677-f249d0b233c1,گلیسیریل منو استئارات,PRD-17014,گلیسرول مونو استئارات یا ایراپون GMS یک ترکیب ...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[گلیسیریل, استئار]","[گلیسرول, مونو, استئار, ایراپون, GMS, ترکیب, ر...",[]
16993,16993,a824fe76-95ca-42f6-9b21-d5522a8a808b,'گلیسیرین تصفیه شده گرید غذایی,PRD-17015,گلیسیرین تصفیه شده گرید غذائی یا ایراسوییت یک ...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[گلیسیرین, گریست#گری]","[گلیسیرین, گریست#گری, ایراسو, امولسیفایر, بو, ...",[]
16994,16994,148a77f2-d054-4a45-abd9-28e0b993764e,کوکامیدوپروپیل بتائین (اکتیو ۳۰ ٪),PRD-17016,کوکامیدوپروپیل بتائین یک سورفکتانت آمفوتریک بر...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[کوکامیدوپروپیل, بتائین, اکتیو]","[کوکامیدوپروپیل, بتائین, سورفکتان, آمفوتریک, پ...",[]
16995,16995,9c2b9a60-f5b9-41be-b3d4-dd823e6271f0,کوکامیدوپروپیل بتائین(اکتیو ۳۷٪),PRD-17017,آمفونیل C۴۵ کوکامیدوپروپیل بتائین با اکتیو (با...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[کوکامیدوپروپیل, بتائین, اکتیو]","[آمفونیل, کوکامیدوپروپیل, بتائین, اکتیو, سورفک...",[]
16996,16996,07902566-4459-47f5-8368-4605fe322682,آلکیل دی متیل آمین اکساید,PRD-17019,آلکیل دی متیل آمین اکساید یا اکسونیل LC مایعی ...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[آلکیل, د, متیل, آمین, اکساید]","[آلکیل, د, متیل, آمین, اکساید, اکسونیل, LC, ما...",[]
16997,16997,0c2e2acf-3c0b-478c-ab5a-6ad4227b92af,بیس (آسیلوکسی اتیل) آمونیوم استات,PRD-17020,بیس آسیلوکسی (تیل آمونیوم استات یا ایراسافت ،S...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[بیس, آسیلوکس, اتیل, آمونیو, اس]","[بیس, آسیلوکس, تیل, آمونیو, اس, ایراساف, SD, س...",[]
16998,16998,e59ca28c-0911-4cce-a30d-dc7ff8eca784,بیس(آسیلوکسی اتیل) هیدروکسی اتیل متیل آمونیوم ...,PRD-17021,بیس آسیلوکسی (تیل هیدروکسی اتیل متیل آمونیوم م...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[بیس, آسیلوکس, اتیل, هیدروکس, اتیل, متیل, آمون...","[بیس, آسیلوکس, تیل, هیدروکس, اتیل, متیل, آمونی...",[]
16999,16999,37c44528-d833-4487-8c53-8249c0574a59,بیس(آسیلوکسی اتیل) هیدروکسی اتیل متیل آمونیوم ...,PRD-17022,بیس (آسیلوکسی (تیل (هیدروکسی اتیل متیل آمونیوم...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[بیس, آسیلوکس, اتیل, هیدروکس, اتیل, متیل, آمون...","[بیس, آسیلوکس, تیل, هیدروکس, اتیل, متیل, آمونی...",[]
17000,17000,f35630ac-36c0-4314-a02f-33b2323583de,پالم کرنل فتی اسید دی اتانول آمید,PRD-17023,پالم کرنل فتی اسید دی اتانول آمید یا ایرامید ،...,[],c9d557f1-e90a-4fe6-be67-65a11f8e51a5,COMPANY,لابراتوار دکتر اخوی,"[پال, کرنل, فت, اسید, د, اتانول]","[پال, کرنل, فت, اسید, د, اتانول, ایرامید, PKD,...",[]


In [100]:
my_list= tokenized_dmd_df['tokenized_dmd_description'].sum()
count_dict = Counter(my_list)
# Find the top most common values
top_common_values = count_dict.most_common(1000)
df_top_common_dmd = pd.DataFrame(top_common_values, columns=['Value', 'Count'])
df_top_common_dmd.to_csv('top_common_values_dmd.csv', index=False)

In [101]:
my_list= tokenized_prd_df['tokenized_prd_key_words'].sum()
count_dict = Counter(my_list)
# Find the top most common values
top_common_values = count_dict.most_common(1000)
df_top_common_prd = pd.DataFrame(top_common_values, columns=['Value', 'Count'])
df_top_common_prd.to_csv('top_common_values_prd.csv', index=False)